In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model


import datetime

import numpy as np

from plot import vessel_planning

from vo_colos.utils.object_registry import ClassInspect

"""File that defines the Coloses."""
import inspect
import logging
from typing import ClassVar, Dict

from numpydoc.docscrape import NumpyDocString

In [4]:
class ActivityRegistry(ClassInspect):
    """Abstract Base Class for the OpenCLSim Sites."""

    # task registry
    registry: ClassVar[Dict[str, "SiteRegistry"]] = {}

    # task identification key
    key: str

    def __init_subclass__(cls, *args, **kwargs):
        """
        Register all classes that inherit from the `BaseTask` class.

        All child classes are stored in a class property `registry` under a
        identification key that is task-specific. The task-specific key must be stored
        in the class property `key` and must be a string. The task-specific key can be
        used as value for the `engine` property of a job.

        """
        if not cls.key:
            logger.warning("Task key is not defined.")
        elif not isinstance(cls.key, str):
            logger.warning(f"Task key is not a string: {cls.key}.")
        elif cls.key in cls.registry.items():
            logger.warning(f"Task with key '{cls.key}' already defined.'")
        else:
            cls.registry[str(cls.key)] = cls
        super().__init_subclass__(*args, **kwargs)

    def __init__(self, *args, **kwargs):
        """Init of the task class."""
        super().__init__(*args, **kwargs)

class NewMoveActivity(ActivityRegistry, model.MoveActivity):
    key = "MoveActivity"
class NewBasicActivity(model.BasicActivity, ActivityRegistry):
    key = "BasicActivity"
class NewShiftAmountActivity(model.ShiftAmountActivity, ActivityRegistry):
    key = "ShiftAmountActivity"
class NewWhileActivity(model.WhileActivity, ActivityRegistry):
    key = "WhileActivity"
class NewSequentialActivity(model.SequentialActivity, ActivityRegistry):
    key = "SequenceActivity"

In [5]:
ActivityRegistry.registry

{'MoveActivity': __main__.NewMoveActivity,
 'BasicActivity': __main__.NewBasicActivity,
 'ShiftAmountActivity': __main__.NewShiftAmountActivity,
 'WhileActivity': __main__.NewWhileActivity,
 'SequenceActivity': __main__.NewSequentialActivity}

In [6]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)


TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

In [7]:
activities = [
    {
        "name": "SingeRunCycle1",
        "class": "WhileActivity",
        "children": [
            "WhileFull1"
        ],
        "properties": {
        "condition_event": [
            {"type":"container", "concept": "to_site", "state":"full"}],
        }
    },
    {
        "name": "WhileFull1",
        "class": "SequenceActivity",
        "children": [
              "sailing_empty1",
              "loading1",
              "sailing_full1",
              "unloading1"
          ],
        "properties": {}
    },
    {
        "name": "sailing_empty1",
        "class": "MoveActivity",
        "properties":{
            "mover": "barge_1",
            "destination": "from_site",
        }
    },
    {
        "name": "loading1",
        "class": "ShiftAmountActivity",
        "properties":{
            "processor": "cutter",
            "origin": "from_site",
            "destination": "barge_1",
            "amount": 5,
            "duration": 3600,
        }
    },
    {
        "name": "sailing_full1",
        "class": "MoveActivity",
        "properties":{
            "mover": "barge_1",
            "destination": "to_site",
        }
    },
    {
        "name": "unloading1",
        "class": "ShiftAmountActivity",
        "properties":{
            "processor": "barge_1",
            "origin": "barge_1",
            "destination": "to_site",
            "amount": 5,
            "duration": 3600,
        }
    },
    
    {
        "name": "SingeRunCycle",
        "class": "WhileActivity",
        "children": [
            "WhileFull"
        ],
        "properties": {
        "condition_event": [
            {"type":"container", "concept": "to_site", "state":"full"}],
        }
    },
    {
        "name": "WhileFull",
        "class": "SequenceActivity",
        "children": [
              "sailing_empty",
              "loading",
              "sailing_full",
              "unloading"
          ],
        "properties": {}
    },
    {
        "name": "sailing_empty",
        "class": "MoveActivity",
        "properties":{
            "mover": "barge_2",
            "destination": "from_site",
        }
    },
    {
        "name": "loading",
        "class": "ShiftAmountActivity",
        "properties":{
            "processor": "cutter",
            "origin": "from_site",
            "destination": "barge_2",
            "amount": 5,
            "duration": 3600,
        }
    },
    {
        "name": "sailing_full",
        "class": "MoveActivity",
        "properties":{
            "mover": "barge_2",
            "destination": "to_site",
        }
    },
    {
        "name": "unloading",
        "class": "ShiftAmountActivity",
        "properties":{
            "processor": "barge_2",
            "origin": "barge_2",
            "destination": "to_site",
            "amount": 5,
            "duration": 3600,
        }
    },
]

In [8]:
ActivityRegistry.registry

{'MoveActivity': __main__.NewMoveActivity,
 'BasicActivity': __main__.NewBasicActivity,
 'ShiftAmountActivity': __main__.NewShiftAmountActivity,
 'WhileActivity': __main__.NewWhileActivity,
 'SequenceActivity': __main__.NewSequentialActivity}

In [9]:
ActivityRegistry.inspect("WhileActivity")

{'type': 'object',
 'properties': {'sub_process': {'type': 'null'},
  'condition_event': {'type': 'null'},
  'show': {'type': 'null'},
  'registry': {'type': 'null'},
  'postpone_start': {'type': 'null'},
  'start_event': {'type': 'null'},
  'requested_resources': {'type': 'null'},
  'keep_resources': {'type': 'null'},
  'name': {'type': 'null'},
  'ID': {'type': 'null'},
  'env': {'type': 'null'}},
 'required': ['condition_event', 'name', 'registry', 'env', 'sub_process']}

In [10]:
class SimulationFactory:
    def __init__(self, activities):
        self.activities = activities
    
    def restfactory(self):
        simulation_start = datetime.datetime.now()

        self.my_env = simpy.Environment(initial_time=time.mktime(simulation_start.timetuple()))
        self.my_env.epoch = time.mktime(simulation_start.timetuple())

        location_from_site = shapely.geometry.Point(5.1, 52)
        location_to_site = shapely.geometry.Point(5, 52.1)
        location_to_site_2 = shapely.geometry.Point(5, 52.2)

        data_from_site = {
            "env": self.my_env,
            "name": "from_site",
            "geometry": location_from_site,
            "capacity": 110,
            "level": 100,
        }

        data_to_site = {
            "env": self.my_env,
            "name": "to_site",
            "geometry": location_to_site,
            "capacity": 55,
            "level": 0,
        }

        data_to_site_2 = {
            "env": self.my_env,
            "name": "to_site_2",
            "geometry": location_to_site_2,
            "capacity": 55,
            "level": 0,
        }

        data_cutter = {
            "env": self.my_env,
            "name": "cutter",
            "geometry": location_from_site,
            "capacity": 5,
            "compute_v": lambda x: 10,
            "loading_rate": 3600/5,
            "unloading_rate": 3600/5
        }


        data_barge_1 = {
            "env": self.my_env,
            "name": "barge_1",
            "geometry": location_to_site,
            "capacity": 5,
            "compute_v": lambda x: 10,
            "loading_rate": 3600/5,
            "unloading_rate": 3600/5
        }


        data_barge_2 = {
            "env": self.my_env,
            "name": "barge_2",
            "geometry": location_to_site,
            "capacity": 5,
            "compute_v": lambda x: 10,
            "loading_rate": 3600/5,
            "unloading_rate": 3600/5
        }

        self.simulation_objects = {
            "vessels":[
                TransportProcessingResource(**data_cutter),
                TransportProcessingResource(**data_barge_1),
               TransportProcessingResource(**data_barge_2),
                ],
            "sites":[
                Site(**data_from_site),
                Site(**data_to_site),
                Site(**data_to_site_2),
                ],
            "activities": []
        }
        
    def get_from_list(self, name, array):
        obj = [a for a in array if a.name == name]
        assert len(obj) == 1
        return obj[0]
        
    
    def build_activity(self, act):       
        if act["class"] == "ShiftAmountActivity":
            objects = list(self.simulation_objects['sites'])
            objects.extend(self.simulation_objects['vessels'])
            
            processor = self.get_from_list(act["properties"]["processor"], self.simulation_objects['vessels'])
            origin = self.get_from_list(act["properties"]["origin"], objects)
            destination = self.get_from_list(act["properties"]["destination"], objects)
            
            print(objects, act["properties"]["destination"])
            assert processor is not None
            assert origin is not None
            assert destination is not None
            
            obj= model.ShiftAmountActivity(
                name=act["name"],
                env= self.my_env,
                registry = self.registry,
                requested_resources=self.requested_resources,
                processor=processor,
                origin=origin,
                destination=destination,
                amount=act["properties"]["amount"],
                duration=act["properties"]["duration"],
                postpone_start=act["properties"]["postpone_start"],
            )
            return obj
        
        if act["class"] == "MoveActivity":
            mover = self.get_from_list(act["properties"]["mover"], self.simulation_objects['vessels'])
            destination=self.get_from_list(act["properties"]["destination"], self.simulation_objects['sites'])
            
            assert mover is not None
            assert destination is not None
            
            obj= model.MoveActivity(
                name=act["name"],
                env= self.my_env,
                registry = self.registry,
                requested_resources=self.requested_resources,
                mover=mover,
                destination=destination,
                postpone_start=act["properties"]["postpone_start"],
            )
            return obj
        
        if act["class"] == "WhileActivity":
            sub = [a for a in self.simulation_objects["activities"] if a.name in act["children"]] 
            assert len(sub) == 1
            
            condition_event=act["properties"]["condition_event"]
            for event in range(len(condition_event)):
                concept = condition_event[event].get("concept")
                concept = self.get_from_list(concept, self.simulation_objects["sites"])
                assert concept is not None
                condition_event[event]["concept"] = concept
                
            print(condition_event)
            
            obj= model.WhileActivity(
                name=act["name"],
                env= self.my_env,
                registry = self.registry,
                requested_resources=self.requested_resources,
                postpone_start=act["properties"]["postpone_start"],
                condition_event=act["properties"]["condition_event"],
                sub_process = sub[0]
            )
            return obj
        

        if act["class"] == "SequenceActivity":
            sub = [a for a in self.simulation_objects["activities"] if a.name in act["children"]]            
            obj= model.SequentialActivity(
                name=act["name"],
                env= self.my_env,
                registry = self.registry,
                requested_resources=self.requested_resources,
                postpone_start=act["properties"]["postpone_start"],
                sub_processes=sub,
            )
            return obj
                
    def activity_factory(self):
        basic_activites = ["MoveActivity", "ShiftAmountActivity", "BasicActivity"]
        self.registry = {}
        self.requested_resources = {}
        
        total_children = []
        for child in [act.get("children", []) for act in self.activities]:
            total_children.extend(child)

        for act in self.activities:
            if act["class"] in basic_activites:
                act["properties"]["postpone_start"] = act["name"] in total_children
                self.simulation_objects["activities"].append(self.build_activity(act))
        
        while len(self.simulation_objects["activities"]) != len(self.activities):
            for act in self.activities:
                activity_names = [a.name for a in self.simulation_objects["activities"]] 
                dependencies = [c in activity_names for c in act.get("children", [])]
                
                if (
                    act["name"] not in activity_names and 
                    False not in dependencies
                ):
                    act["properties"]["postpone_start"] = act["name"] in total_children
                    self.simulation_objects["activities"].append(self.build_activity(act))
        
                
    def run(self):
        self.restfactory()
        self.activity_factory()
        
        self.my_env.run()

In [11]:
sim = SimulationFactory(activities)
sim.run()

init
level: 0
completed init
init
level: 0
completed init
init
level: 0
completed init
init
level: 100
completed init
init
level: 0
completed init
init
level: 0
completed init
[<__main__.Site object at 0x000001F35597BE88>, <__main__.Site object at 0x000001F355991388>, <__main__.Site object at 0x000001F355997CC8>, <__main__.TransportProcessingResource object at 0x000001F35597B3C8>, <__main__.TransportProcessingResource object at 0x000001F354654C48>, <__main__.TransportProcessingResource object at 0x000001F35597B7C8>] barge_1
[<__main__.Site object at 0x000001F35597BE88>, <__main__.Site object at 0x000001F355991388>, <__main__.Site object at 0x000001F355997CC8>, <__main__.TransportProcessingResource object at 0x000001F35597B3C8>, <__main__.TransportProcessingResource object at 0x000001F354654C48>, <__main__.TransportProcessingResource object at 0x000001F35597B7C8>] to_site
[<__main__.Site object at 0x000001F35597BE88>, <__main__.Site object at 0x000001F355991388>, <__main__.Site object a

release destination : {<simpy.resources.resource.Resource object at 0x000001F35597B988>: <Request() object at 0x1f35583b988>, <simpy.resources.resource.Resource object at 0x000001F35597B9C8>: <Request() object at 0x1f35583b9c8>}
released origin : {<simpy.resources.resource.Resource object at 0x000001F35597B9C8>: <Request() object at 0x1f35583b9c8>}
released processor : {<simpy.resources.resource.Resource object at 0x000001F35597B9C8>: <Request() object at 0x1f35583b9c8>}
{<simpy.resources.resource.Resource object at 0x000001F35597B9C8>: <Request() object at 0x1f35583b9c8>}
Activity end(unloading)
put_callback - id_ default
{'default': {55: <Event() object at 0x1f355982a88>}}
amount :55
destination store after put: [{'id': 'default', 'level': 30, 'capacity': 55}]
after put
processor end process
Processed 5 of default:
  by:          barge_1
  origin        barge_1  contains: 0 of default
  destination:  to_site contains: 30 of default
after shift amount : {<simpy.resources.resource.Reso

In [12]:
objects = list(sim.simulation_objects["vessels"])
activities = []
for obj in objects:
    print(set(obj.log["Message"]))
    activities.extend(set(obj.log["Message"]))

activities = list(set(activities))

            
C = np.linspace(0,255, len(activities))
colors = {}

for i in range(len(activities)):
    colors[i] = f'rgb({int(C[i]/2)},{int(C[len(C) - 1 - i]/1.5)},{int(C[i])})'

vessel_planning(objects, activities, colors)

{'transfer default to barge_2', 'transfer default to barge_1'}
{'sailing empty', 'transfer default to to_site', 'sailing filled', 'transfer default to barge_1'}
{'sailing empty', 'transfer default to barge_2', 'transfer default to to_site', 'sailing filled'}
